In [77]:
import numpy
import pandas as pd
from glob import glob

In [142]:
def compute_daily_slope(temp_file):
    temps = pd.read_csv(temp_file)
    # 1am (hot) to 11am (cold) is usually the most dramatic "warmest to coldest" temp differential.
    # computing the slope between those two times gives a "per hour temperature change" value
    one_oclocks = temps.where(temps['Time'] == '01:00').dropna(how='all').add_prefix('01_').reset_index()
    eleven_oclocks = temps.where(temps['Time'] == '11:00').dropna(how='all').add_prefix('11_').reset_index()
    df = pd.concat([one_oclocks, eleven_oclocks], axis=1)
    df['slopes'] = (df['11_avg(temp)'] - df['01_avg(temp)']) / (11-1) #10 hours between 11 and 1
    return df

In [ ]:
def compute_monthly_averages(files):
    # monthly averages are fun to look at
    for f in files:
        try:
            new = compute_daily_slope(f)
            monthly_slope = new['slopes'].mean()
            print(f.split('/')[1], '-'.join(f.split('/')[2:4]), monthly_slope)
        except pd.io.common.EmptyDataError:
            continue

In [ ]:
def combine_all_files(files):
    # combines all the nest data files into one big file so we can really cripple your computer
    dfs = [pd.read_csv(f, index_col=[0], parse_dates=[0]) for f in files]
    return pd.concat(dfs, axis=0, join='outer').sort_index()

In [186]:
def output_plottable_daily(files):
    # outputs a dataframe with only "Date" and "slope" so you can plot over time
    dailies = [compute_daily_slope(f) for f in files]
    f_dailies = pd.concat(dailies, axis=0, join='outer').sort_index()
    return f_dailies[['01_Date', 'slopes']]

In [188]:
# assuming your Nest export data files are in standard google format and next to this notebook
files = glob('thermostats/*/*/*/*.csv')
df = output_plottable_daily(files)
df.describe()
#df.to_csv('some/path/file.csv')

,slopes
count,1068.000000
mean,-0.138516
std,0.104437
min,-0.594000
25%,-0.198000
50%,-0.134000
75%,-0.062000
max,0.179000
